## Improving the strategy

[The previous notebook](https://github.com/timk11/bitcoin-trading-strategies/blob/master/BTC_spot_strategy.ipynb) introduced a bitcoin trading strategy based on simply checking the market once a day and buying or selling when the price drops by 3% from the highest high since the last buy, or rises by 3% from the lowest low since the last sell. We found that the inclusion of stop orders was necessary for the strategy to be fruitful.

But what if we changed the target percentages? I chose 3% by eyeballing historical charts and making a best guess as to how this strategy might work. This notebook aims to improve the strategy by testing out 25 combinations of "rise" and "fall" targets ranging from 1% to 5%. "Rise" is the percentage by which the price should rise above the lowest low since the last sell in order to trigger a buy. "Fall" is the percentage by which the price should fall below the highest high since the last buy in order to trigger a sell.

In [1]:
import pandas as pd
df1 = pd.read_csv("BTC-USD 200212.csv")
df1 = df1.drop("Adj Close", axis=1)
df1 = df1.iloc[:-1]

In [2]:
df1["new_high"] = [0] * len(df1.index)
df1["new_low"] = [0] * len(df1.index)
df1["stop_val"] = [0] * len(df1.index)
df1["BTC_held"] = [1.0] * len(df1.index)
df1["USD_held"] = [0] * len(df1.index)
df1["USD_val"] = [0] * len(df1.index)
df1["BTC_val"] = [1.0] * len(df1.index)
df1["action"] = [""] * len(df1.index)

In [3]:
for rise in range(1,6):
    for fall in range(1,6):
        df = df1
        df.loc[df.index[0], "new_high"] = df.loc[df.index[0], "High"]
        df.loc[df.index[0], "new_low"] = df.loc[df.index[0], "Low"]
        df.loc[df.index[0], "stop_val"] = df.loc[df.index[0], "Open"] * (1 - fall/100)
        df.loc[df.index[0], "USD_val"] = df.loc[df.index[0], "Close"]
        df.loc[df.index[0], "new_high"] = df.loc[df.index[0], "High"]
        df.loc[df.index[0], "new_low"] = df.loc[df.index[0], "Low"]
        
        if df.loc[df.index[0], "Low"] <= df.loc[df.index[0], "stop_val"]:
            df.loc[df.index[0], "action"] = "SELL @ " + str(int(df.loc[df.index[0], "stop_val"]))
            df.loc[df.index[0], "USD_held"] = df.loc[df.index[0], "stop_val"]
            df.loc[df.index[0], "BTC_held"] = 0
            df.loc[df.index[0], "USD_val"] = df.loc[df.index[0], "USD_held"]
            df.loc[df.index[0], "BTC_val"] = df.loc[df.index[0], "USD_val"] / df.loc[df.index[0], "Close"]
        
        for i in range(1, len(df.index)):
            if df.loc[df.index[i-1], "BTC_held"] > df.loc[df.index[i-1], "USD_held"]:
                df.loc[df.index[i], "USD_val"] = df.loc[df.index[i-1], "BTC_held"] * df.loc[df.index[i], "Close"]
                df.loc[df.index[i], "BTC_val"] = df.loc[df.index[i-1], "BTC_val"]
                df.loc[df.index[i], "stop_val"] = df.loc[df.index[i-1], "new_high"] * (1 - fall/100)
                if df.loc[df.index[i], "Open"] <= df.loc[df.index[i], "stop_val"]:
                    df.loc[df.index[i], "action"] = "SELL @ " + str(int(df.loc[df.index[i], "Open"]))
                    df.loc[df.index[i], "new_low"] = df.loc[df.index[i], "Low"]
                    df.loc[df.index[i], "USD_held"] = df.loc[df.index[i-1], "BTC_held"] * df.loc[df.index[i], "Open"]
                    df.loc[df.index[i], "BTC_held"] = 0
                    df.loc[df.index[i], "USD_val"] = df.loc[df.index[i], "USD_held"]
                    df.loc[df.index[i], "BTC_val"] = df.loc[df.index[i], "USD_val"] / df.loc[df.index[i], "Close"]
                elif df.loc[df.index[i], "Low"] <= df.loc[df.index[i], "stop_val"]:
                    df.loc[df.index[i], "action"] = "SELL @ " + str(int(df.loc[df.index[i], "stop_val"]))
                    df.loc[df.index[i], "new_low"] = df.loc[df.index[i], "Low"]
                    df.loc[df.index[i], "USD_held"] = df.loc[df.index[i-1], "BTC_held"] * df.loc[df.index[i], "stop_val"]
                    df.loc[df.index[i], "BTC_held"] = 0
                    df.loc[df.index[i], "USD_val"] = df.loc[df.index[i], "USD_held"]
                    df.loc[df.index[i], "BTC_val"] = df.loc[df.index[i], "USD_val"] / df.loc[df.index[i], "Close"]
                elif df.loc[df.index[i], "High"] > df.loc[df.index[i-1], "new_high"]:
                    df.loc[df.index[i], "BTC_held"] = df.loc[df.index[i-1], "BTC_held"]
                    df.loc[df.index[i], "USD_held"] = df.loc[df.index[i-1], "USD_held"]
                    df.loc[df.index[i], "new_high"] = df.loc[df.index[i], "High"]
                else:
                    df.loc[df.index[i], "BTC_held"] = df.loc[df.index[i-1], "BTC_held"]
                    df.loc[df.index[i], "USD_held"] = df.loc[df.index[i-1], "USD_held"]
                    df.loc[df.index[i], "new_high"] = df.loc[df.index[i-1], "new_high"]
            
            else:
                df.loc[df.index[i], "BTC_val"] = df.loc[df.index[i-1], "USD_held"] / df.loc[df.index[i], "Close"]
                df.loc[df.index[i], "USD_val"] = df.loc[df.index[i-1], "USD_val"]
                df.loc[df.index[i], "stop_val"] = df.loc[df.index[i-1], "new_low"] * (1 + rise/100)
                if df.loc[df.index[i], "Open"] >= df.loc[df.index[i], "stop_val"]:
                    df.loc[df.index[i], "action"] = "BUY @ " + str(int(df.loc[df.index[i], "Open"]))
                    df.loc[df.index[i], "new_high"] = df.loc[df.index[i], "High"]
                    df.loc[df.index[i], "BTC_held"] = df.loc[df.index[i-1], "USD_held"] / df.loc[df.index[i], "Open"]
                    df.loc[df.index[i], "USD_held"] = 0
                    df.loc[df.index[i], "BTC_val"] = df.loc[df.index[i], "BTC_held"]
                    df.loc[df.index[i], "USD_val"] = df.loc[df.index[i], "BTC_val"] * df.loc[df.index[i], "Close"]
                elif df.loc[df.index[i], "High"] >= df.loc[df.index[i], "stop_val"]:
                    df.loc[df.index[i], "action"] = "BUY @ " + str(int(df.loc[df.index[i], "stop_val"]))
                    df.loc[df.index[i], "new_high"] = df.loc[df.index[i], "High"]
                    df.loc[df.index[i], "BTC_held"] = df.loc[df.index[i-1], "USD_held"] / df.loc[df.index[i], "stop_val"]
                    df.loc[df.index[i], "USD_held"] = 0
                    df.loc[df.index[i], "BTC_val"] = df.loc[df.index[i], "BTC_held"]
                    df.loc[df.index[i], "USD_val"] = df.loc[df.index[i], "BTC_val"] * df.loc[df.index[i], "Close"]
                elif df.loc[df.index[i], "Low"] < df.loc[df.index[i-1], "new_low"]:
                    df.loc[df.index[i], "BTC_held"] = df.loc[df.index[i-1], "BTC_held"]
                    df.loc[df.index[i], "USD_held"] = df.loc[df.index[i-1], "USD_held"]
                    df.loc[df.index[i], "new_low"] = df.loc[df.index[i], "Low"]
                else:
                    df.loc[df.index[i], "BTC_held"] = df.loc[df.index[i-1], "BTC_held"]
                    df.loc[df.index[i], "USD_held"] = df.loc[df.index[i-1], "USD_held"]
                    df.loc[df.index[i], "new_low"] = df.loc[df.index[i-1], "new_low"]
                    
        print("With a rise limit of " + str(rise) +"%, and a fall limit of " + str(fall) + "%,")
        print("         USD value of holdings changed from " + str(round((df.loc[df.index[0], "USD_val"]), 2)) + " to " + str(round((df.loc[df.index[1972], "USD_val"]), 2)) + ", a change of " + str(round(((df.loc[df.index[1972], "USD_val"] / df.loc[df.index[0], "USD_val"] - 1) * 100), 1)) + "%,")
        print("         BTC value of holdings changed from " + str(round((df.loc[df.index[0], "BTC_val"]), 2)) + " to " + str(round((df.loc[df.index[1972], "BTC_val"]), 2)) + ", a change of " + str(round(((df.loc[df.index[1972], "BTC_val"] / df.loc[df.index[0], "BTC_val"] - 1) * 100), 1)) + "%,")
        print("         and BTC price changed from " + str(round((df.loc[df.index[0], "Close"]), 2)) + " to " + str(round((df.loc[df.index[1972], "Close"]), 2)) + ", a change of " + str(round(((df.loc[df.index[1972], "Close"] / df.loc[df.index[0], "Close"] - 1) * 100), 1)) + "%.\n")
        for [yr,j,k] in ([[2015, 105, 470], [2016, 470, 836], [2017, 836, 1201], [2018, 1201, 1566], [2019, 1566, 1931]]):
            print("     " + str(yr) + ": USD value " + str(round(((df.loc[df.index[k], "USD_val"] / df.loc[df.index[j], "USD_val"] - 1) * 100), 1)) + "%, BTC value " + str(round(((df.loc[df.index[k], "BTC_val"] / df.loc[df.index[j], "BTC_val"] - 1) * 100), 1)) + "%, BTC price " + str(round(((df.loc[df.index[k], "Close"] / df.loc[df.index[j], "Close"] - 1) * 100), 1)) + "%\n")

With a rise limit of 1%, and a fall limit of 1%,
         USD value of holdings changed from 461.21 to 53749.37, a change of 11554.1%,
         BTC value of holdings changed from 1.01 to 5.45, a change of 440.7%,
         and BTC price changed from 457.33 to 9856.61, a change of 2055.2%.

     2015: USD value 56.1%, BTC value 16.1%, BTC price 34.5%

     2016: USD value 109.7%, BTC value -6.3%, BTC price 123.8%

     2017: USD value 772.7%, BTC value -40.6%, BTC price 1368.9%

     2018: USD value -18.7%, BTC value 207.4%, BTC price -73.6%

     2019: USD value 266.3%, BTC value 90.6%, BTC price 92.2%

With a rise limit of 1%, and a fall limit of 2%,
         USD value of holdings changed from 456.55 to 94114.43, a change of 20514.4%,
         BTC value of holdings changed from 1.0 to 9.55, a change of 856.5%,
         and BTC price changed from 457.33 to 9856.61, a change of 2055.2%.

     2015: USD value 190.2%, BTC value 115.8%, BTC price 34.5%

     2016: USD value 209.7%, BTC valu

With a rise limit of 3%, and a fall limit of 5%,
         USD value of holdings changed from 457.33 to 302774.23, a change of 66104.2%,
         BTC value of holdings changed from 1.0 to 30.72, a change of 2977.1%,
         and BTC price changed from 457.33 to 9856.61, a change of 2055.2%.

     2015: USD value 226.4%, BTC value 142.7%, BTC price 34.5%

     2016: USD value 183.8%, BTC value 26.8%, BTC price 123.8%

     2017: USD value 1660.2%, BTC value 19.8%, BTC price 1368.9%

     2018: USD value -19.1%, BTC value 205.9%, BTC price -73.6%

     2019: USD value 254.6%, BTC value 84.5%, BTC price 92.2%

With a rise limit of 4%, and a fall limit of 1%,
         USD value of holdings changed from 461.21 to 67682.28, a change of 14575.1%,
         BTC value of holdings changed from 1.01 to 6.87, a change of 580.9%,
         and BTC price changed from 457.33 to 9856.61, a change of 2055.2%.

     2015: USD value 173.4%, BTC value 103.3%, BTC price 34.5%

     2016: USD value 82.6%, BTC 

### Results
The overall winner is the strategy with a "rise" of 3% and a "fall" of 4%, which I'll abbreviate to +3%/-4%. This marginally outperforms the original +3%/-3% strategy to give a 1428-fold increase in US dollar value and a 66-fold increase in bitcoin value of our holding.

Year by year winning strategies with corresponding changes are as follows:

 - **2015: +3%/-4%** - USD value 251.5%, BTC value 161.4%, BTC price 34.5%
 - **2016: +2%/-2%** - USD value 237.0%, BTC value 50.5%, BTC price 123.8%
 - **2017: +1%/-3%** - USD value 3790.3%, BTC value 164.8%, BTC price 1368.9%
 - **2018: +5%/-2%** - USD value 33.4%, BTC value 404.4%, BTC price -73.6%
 - **2019: +2%/-2%** - USD value 422.4%, BTC value 171.8%, BTC price 92.2%
 
Intuitively it makes sense that when price is rising rapidly, as in 2017, a trader would benefit by being fast to buy (small "rise") and slow to sell (large "fall"), and that the opposite would apply when the market is in decline, as in 2018. This shows in these two examples, but overall this pattern is not consistent and there is not enough here to recommend a way of tweaking "rise" and "fall" settings based on the overall market trend.

Although +3%/-4% was the overall best performing strategy, it did produce a slight fall in US dollar holding value in the bear market year of 2018. An unhappy trader risks becoming a undisciplined trader, so perhaps +3%/-3% might have been a more comfortable strategy in terms of consistency, and this one ended up performing better in the last two full years anyway.

Again, *not* financial advice, and all retrospective with nothing to suggest it will work well on future market movements. I've given you lots of numbers if you'd like to play with them, so happy crunching!